In [1]:
import geopandas as gpd
import pysal as ps
from pysal.cg import asShape
import pandas as pd
import numpy as np

In [2]:
df = ps.pdio.read_files(ps.examples.get_path('NAT.shp'))

In [3]:
gdf = gpd.read_file(ps.examples.get_path('NAT.shp'))

In [4]:
from pysal.cg.shapes import _geoJSON_type_to_Pysal_type
def te_asShape(obj):
    """
    Returns a pysal shape object from obj.
    obj must support the __geo_interface__.
    """
    if type(obj) in _geoJSON_type_to_Pysal_type.values():
        # noop if already pysal type
        return obj
    try:
        geo = obj.__geo_interface__
    except AttributeError:
        geo = obj
    if hasattr(geo, 'type'):
        raise TypeError('%r does not appear to be a shape object' % (obj))
    geo_type = geo['type'].lower()
    #if geo_type.startswith('multi'):
    #    raise NotImplementedError, "%s are not supported at this time."%geo_type
    if geo_type in _geoJSON_type_to_Pysal_type:
        return _geoJSON_type_to_Pysal_type[geo_type].__from_geo_interface__(geo)
    else:
        raise NotImplementedError(
            "%s is not supported at this time." % geo_type)

        
def shp2ps(obj):
    gi = obj.__geo_interface__
    t = gi['type'].lower()
    return (_geoJSON_type_to_Pysal_type[t].__from_geo_interface__(gi))

In [5]:
shapelys = gdf.geometry.tolist()
pysals= df.geometry.tolist()
dicts = [x.__geo_interface__ for x in pysals]

In [6]:
%timeit te_asShape(shapelys[np.random.randint(0,3085)])
%timeit asShape(shapelys[np.random.randint(0,3085)])
%timeit shp2ps(shapelys[np.random.randint(0,3085)])

10000 loops, best of 3: 180 µs per loop
10000 loops, best of 3: 179 µs per loop
1000 loops, best of 3: 174 µs per loop


In [7]:
%timeit te_asShape(pysals[np.random.randint(0,3085)])
%timeit asShape(pysals[np.random.randint(0,3085)])
%timeit shp2ps(pysals[np.random.randint(0,3085)])

The slowest run took 18.79 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 1.55 µs per loop
The slowest run took 6.70 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 1.64 µs per loop
10000 loops, best of 3: 74.1 µs per loop


In [8]:
%timeit te_asShape(dicts[np.random.randint(0,3085)])
%timeit asShape(dicts[np.random.randint(0,3085)])

10000 loops, best of 3: 72.7 µs per loop
10000 loops, best of 3: 71.9 µs per loop


In [9]:
mixed = shapelys
mixed.extend(pysals)
mixed.extend(dicts)

In [10]:
%timeit te_asShape(mixed[np.random.randint(0,3085*3)])
%timeit asShape(mixed[np.random.randint(0,3085*3)])

10000 loops, best of 3: 84.5 µs per loop
10000 loops, best of 3: 83.8 µs per loop
